In [1]:
import requests
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import statistics
import numpy as np
import warnings
warnings.filterwarnings("ignore")
query = {
    "size":10000,
  "query": {
    "match_all": {}
  }
}
password = "bqIUDWf+4Q*kX2BUh6RX"
host_name = '10.10.6.148'

index_name = "slide_placement"
url = "https://elastic:{}@{}:9200/{}/_search".format(password, host_name, index_name)
response1 = requests.get(url,json = query, verify = '/etc/elasticsearch/certs/http_ca.crt')
response_json1 = json.loads(response1.text)
slide_placement = pd.json_normalize(response_json1['hits']['hits'])
print("slide_placement data acquired successfully")

index_name = "inline_corrections"
url = "https://elastic:{}@{}:9200/{}/_search".format(password, host_name, index_name)
response2 = requests.get(url,json = query, verify = '/etc/elasticsearch/certs/http_ca.crt')
response_json2 = json.loads(response2.text)
inline_corrections = pd.json_normalize(response_json2['hits']['hits'])
print("inline_corrections data acquired successfully")

index_name = "slide_locking"
url = "https://elastic:{}@{}:9200/{}/_search".format(password, host_name, index_name)
response3 = requests.get(url,json = query, verify = '/etc/elasticsearch/certs/http_ca.crt')
response_json3 = json.loads(response3.text)
slide_locking = pd.json_normalize(response_json3['hits']['hits'])
print("slide_locking data acquired successfully")

index_name = "slide_picking_from_scanner"
url = "https://elastic:{}@{}:9200/{}/_search".format(password, host_name, index_name)
response4 = requests.get(url,json = query, verify = '/etc/elasticsearch/certs/http_ca.crt')
response_json4 = json.loads(response4.text)
slide_picking_from_scanner = pd.json_normalize(response_json4['hits']['hits'])
print("slide_picking_from_scanner data acquired successfully")

index_name = "basket_data"
url = "https://elastic:{}@{}:9200/{}/_search".format(password, host_name, index_name)
response5 = requests.get(url,json = query, verify = '/etc/elasticsearch/certs/http_ca.crt')
response_json5 = json.loads(response5.text)
basket_data = pd.json_normalize(response_json5['hits']['hits'])
print("basket_data data acquired successfully")

# basket_data

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


slide_placement data acquired successfully
inline_corrections data acquired successfully
slide_locking data acquired successfully
slide_picking_from_scanner data acquired successfully
basket_data data acquired successfully


In [ ]:
both = pd.merge(slide_placement,inline_corrections,on=["_source.data.slide_id","_source.data.scanner_name",
                      "_source.data.load_identifier","_source.data.row_index","_source.data.col_index",
                      "_source.data.cluster_name"])

both = both.drop_duplicates(subset="_source.data.slide_id",keep="last")
both['row_col'] = (both['_source.data.row_index']+1).astype(str)+"_"+(both['_source.data.col_index']+1).astype(str)
both = both.sort_values(["_source.data.row_index","_source.data.col_index"], ascending = (True, True))
both['date'] = pd.to_datetime(both['_source.data.time_stamp_x']).dt.date
# both['date'] = pd.to_datetime(both['date'])
both['computed_angle'] = both['_source.data.computed_angle']*(180/3.14)
both['slide_height'] = both['_source.data.slide_height_um']/1000
print("The scanners available are:" + str((both['_source.data.scanner_name'].unique())))
# both.dtypes

In [ ]:
a = both[both['_source.data.scanner_name']=='H01JBA11R']
a = a.sort_values(by = ['date'], ascending = True)
a

In [ ]:
# xx = a.drop_duplicates(subset=['date'], keep='last')
xx = a.drop_duplicates(subset=['_source.data.load_identifier'], keep='last')
load = xx['_source.data.load_identifier'].iloc[-1]
print(load)
df2 = a[a['_source.data.load_identifier']==load]
df2['angle_difference'] = round(df2['_source.data.actual_angle'] - df2['computed_angle'],2)
df2 = df2.sort_values(["_source.data.row_index","_source.data.col_index"], ascending = (True, True))
df2

In [ ]:
#####################################################################################

In [8]:
# THICKNESS PLOT

basket_data = basket_data.dropna(subset=['_source.data.row_index','_source.data.col_index'])
basket_data['_source.data.row_index'] = basket_data['_source.data.row_index'].astype(int)
basket_data['_source.data.col_index'] = basket_data['_source.data.col_index'].astype(int)
basket_data['row_col'] = (basket_data['_source.data.row_index']+1).astype(str)+"_"+(basket_data['_source.data.col_index']+1).astype(str)
basket_data = basket_data.sort_values(["_source.data.row_index","_source.data.col_index"], ascending = (True, True))

# lst = ["CS006-1-1647954319260","CS006-2-1647954319268","CS006-3-1647954319272","CS006-4-1647954319277"]
df = basket_data[basket_data['_source.data.scanner_name'] == 'H01JBA11R']
df['date'] = pd.to_datetime(df['_source.data.time_stamp']).dt.date
df = df.sort_values(by = ['date'], ascending = False)
# df.to_excel('/home/adminspin/Desktop/excel.xlsx')
df
# basket_data
# df3 = basket_data[basket_data['_source.data.load_identifier'] == 'CS003-4-1652284116981']
# df3
# print(len(df3))
# fig = px.scatter(df3, y='row_col',x="_source.data.slide_thickness",color="_source.data.scanner_name",marginal_x="histogram")
# fig.update_yaxes(title='')
# fig.update_xaxes(title='')
# fig.update_layout(legend_title="Scanner",title="Slide Thickness Plot",height=800,yaxis1=dict(title="Slot Position"),xaxis1=dict(title="Slide Thickness(mm)"))
# # fig.write_image("image_"+df2['_source.data.scanner_name'].unique()[0]+".png")
# fig.show()

,_index,_id,_score,_source.data.scanner_name,_source.data.scanner_position,_source.data.load_identifier,_source.data.row_index,_source.data.col_index,_source.data.slide_id,_source.data.cluster_name,_source.data.slide_thickness,_source.data.raw_slide_thickness,_source.data.time_stamp,row_col,date
8696,basket_data,qWEmXYEBUb826lraXhQF,1.0,H01JBA11R,4,CS003-4-1655124391454,0,9,8290,CS003,0.010,2.0,2022-06-13T13:00:04,1_10,2022-06-13
8689,basket_data,UGEOXYEBUb826lraOxQ4,1.0,H01JBA11R,4,CS003-4-1655123111705,0,9,8050,CS003,0.010,2.0,2022-06-13T12:33:42,1_10,2022-06-13
8691,basket_data,VGEOXYEBUb826lraVhS0,1.0,H01JBA11R,4,CS003-4-1655123111705,0,11,8052,CS003,0.010,2.0,2022-06-13T12:33:49,1_12,2022-06-13
8690,basket_data,UmEOXYEBUb826lraSBT0,1.0,H01JBA11R,4,CS003-4-1655123111705,0,10,8051,CS003,0.010,2.0,2022-06-13T12:33:46,1_11,2022-06-13
8674,basket_data,X2ESXYEBUb826lrayxTf,1.0,H01JBA11R,4,CS003-4-1655123870937,0,1,8162,CS003,1.658,34.0,2022-06-13T12:38:41,1_2,2022-06-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9809,basket_data,lzQtg4ABHEUkxH1raVyV,1.0,H01JBA11R,4,CS003-4-1651246030205,1,14,525,CS003,1.402,1630.0,2022-04-29T20:20:41,2_15,2022-04-29
9815,basket_data,yzQtg4ABHEUkxH1rc1x2,1.0,H01JBA11R,4,CS003-4-1651246030205,1,15,526,CS003,1.604,1639.0,2022-04-29T20:27:55,2_16,2022-04-29
9816,basket_data,zzQtg4ABHEUkxH1rdFwi,1.0,H01JBA11R,4,CS003-4-1651246030205,1,16,527,CS003,1.442,1630.0,2022-04-29T20:28:18,2_17,2022-04-29
9821,basket_data,-jQtg4ABHEUkxH1rfFyF,1.0,H01JBA11R,4,CS003-4-1651246030205,1,17,528,CS003,1.558,1637.0,2022-04-29T20:36:14,2_18,2022-04-29


In [9]:
df[df['_source.data.load_identifier'] == 'CS003-4-1655127975892']

,_index,_id,_score,_source.data.scanner_name,_source.data.scanner_position,_source.data.load_identifier,_source.data.row_index,_source.data.col_index,_source.data.slide_id,_source.data.cluster_name,_source.data.slide_thickness,_source.data.raw_slide_thickness,_source.data.time_stamp,row_col,date


In [ ]:
# THICKNESS PLOT

basket_data = basket_data.dropna(subset=['_source.data.row_index','_source.data.col_index'])
basket_data['_source.data.row_index'] = basket_data['_source.data.row_index'].astype(int)
basket_data['_source.data.col_index'] = basket_data['_source.data.col_index'].astype(int)
basket_data['row_col'] = (basket_data['_source.data.row_index']+1).astype(str)+"_"+(basket_data['_source.data.col_index']+1).astype(str)
basket_data = basket_data.sort_values(["_source.data.row_index","_source.data.col_index"], ascending = (True, True))

# lst = ["CS006-1-1647954319260","CS006-2-1647954319268","CS006-3-1647954319272","CS006-4-1647954319277"]

df3 = basket_data[basket_data['_source.data.load_identifier'] == load]
print(len(df3))
fig = px.scatter(df3, y='row_col',x="_source.data.slide_thickness",color="_source.data.scanner_name",marginal_x="histogram")
fig.update_yaxes(title='')
fig.update_xaxes(title='')
fig.update_layout(legend_title="Scanner",title="Slide Thickness Plot",height=800,yaxis1=dict(title="Slot Position"),xaxis1=dict(title="Slide Thickness(mm)"))
# fig.write_image("image_"+df2['_source.data.scanner_name'].unique()[0]+".png")
fig.show()

In [ ]:
basket_data[basket_data['_source.data.load_identifier'] == 'CS005-2-1652453461104'].to_csv('basket.csv')
inline_corrections[inline_corrections['_source.data.load_identifier'] == 'CS005-2-1652453461104'].to_csv('inline.csv')
slide_placement[slide_placement['_source.data.load_identifier'] == 'CS005-2-1652453461104'].to_csv('placement.csv')
slide_locking[slide_locking['_source.data.load_identifier'] == 'CS005-2-1652453461104'].to_csv('locking.csv')
slide_picking_from_scanner[slide_picking_from_scanner['_source.data.load_identifier'] == 'CS005-2-1652453461104'].to_csv('picking.csv')

In [ ]:
# INCOMING ANGLE
# print(len(df2))
fig = make_subplots(
            rows=1, cols=4,
            subplot_titles=("<b>Row : 1 ","<b>Row : 2 ","<b>Row : 3 ","<b>Row : 4 "))
for i in range(4):
    fig.add_trace(go.Scatter(x=df2[df2['_source.data.row_index']==i]['_source.data.actual_angle'],y=df2[df2['_source.data.row_index']==i]['row_col'],
                             name="Actual Angle",showlegend=False,mode="markers",marker=dict(color="MediumPurple")),row=1,col=(i+1))
    fig.add_trace(go.Scatter(x=df2[df2['_source.data.row_index']==i]['computed_angle'],y=df2[df2['_source.data.row_index']==i]['row_col'],
                             name="Computed Angle",showlegend=False,mode="markers",marker=dict(color="salmon")),row=1,col=(i+1))
    fig.add_annotation(y=-3,text="<b>Postive Adjustments : "+str(len(df2[(df2['_source.data.row_index']==i)&(df2['angle_difference']>0)]))+\
                   "<br>Negative Adjustments : "+str(len(df2[(df2['_source.data.row_index']==i)&(df2['angle_difference']<0)]))+\
                       "<br>μ Postive Adjustments : "+str(round(np.mean(df2[(df2['_source.data.row_index']==i)&(df2['angle_difference']>0)]['angle_difference']),2))+\
                       "<br>μ Negative Adjustments : "+str(round(np.mean(df2[(df2['_source.data.row_index']==i)&(df2['angle_difference']<0)]['angle_difference']),2)),
                       showarrow=False,row=1,col=(i+1))

fig.add_trace(go.Scatter(x=df2[df2['_source.data.row_index']==0]['computed_angle'],y=df2[df2['_source.data.row_index']==0]['row_col'],
                         name="Incoming Angle",mode="markers",marker=dict(color="salmon")),row=1,col=1)
fig.add_trace(go.Scatter(x=df2[df2['_source.data.row_index']==0]['_source.data.actual_angle'],y=df2[df2['_source.data.row_index']==0]['row_col'],
                         name="Corrected Angle",mode="markers",marker=dict(color="MediumPurple")),row=1,col=1)
fig.update_layout(title="<b>Angle Adjustment Plot for "+df2['_source.data.scanner_name'].unique()[0]+" "+str(df2['date'].unique()[-1]),height=800,width=1400)
fig.update_yaxes(autorange="reversed")
fig.update_yaxes(title="Slot Position",row=1,col=1)
fig.update_xaxes(title="Slide Angle")
fig.update_xaxes(range=[-4.3,4.3])
fig.add_vline(x=-4, line_width=3, line_dash="dash", line_color="red")
fig.add_vline(x=4, line_width=3, line_dash="dash", line_color="red")
fig.add_annotation(x=1.14,y=0.9,xref="paper",yref="paper",
        text="<br><b>Basket Level Details <br>"+"<b>+ve Adjustments : "+str(len(df2[df2['angle_difference']>0]))+\
                   "<br>-ve Adjustments : "+str(len(df2[df2['angle_difference']<0]))+\
                       "<br>μ +ve Adjustments : "+str(round(np.mean(df2[df2['angle_difference']>0]['angle_difference']),2))+\
                       "<br>μ -ve Adjustments :"+str(round(np.mean(df2[df2['angle_difference']<0]['angle_difference']),2)),
        showarrow=False,font=dict(family="Courier New, monospace",size=10,color="black"),align="center",
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="white",
        opacity=0.8
        )
fig.show()


# X OFFSET


fig = make_subplots(
            rows=1, cols=4,
            subplot_titles=("<b>Row : 1 ","<b>Row : 2 ","<b>Row : 3 ","<b>Row : 4 "))
for i in range(4):
    fig.add_trace(go.Scatter(x=df2[df2['_source.data.row_index']==i]['_source.data.offset_pos_x_um'],y=df2[df2['_source.data.row_index']==i]['row_col'],
                             name="X-Offset",showlegend=False,mode="markers",marker=dict(color="MediumPurple")),row=1,col=(i+1))
    
    fig.add_annotation(y=-3,text="<b>Postive Adjustments : "+str(len(df2[(df2['_source.data.row_index']==i)&(df2['_source.data.offset_pos_x_um']>0)]))+\
                   "<br>Negative Adjustments : "+str(len(df2[(df2['_source.data.row_index']==i)&(df2['_source.data.offset_pos_x_um']<0)]))+\
                       "<br>μ Postive Adjustments : "+str(round(np.mean(df2[(df2['_source.data.row_index']==i)&(df2['_source.data.offset_pos_x_um']>0)]['_source.data.offset_pos_x_um']),2))+\
                       "<br>μ Negative Adjustments : "+str(round(np.mean(df2[(df2['_source.data.row_index']==i)&(df2['_source.data.offset_pos_x_um']<0)]['_source.data.offset_pos_x_um']),2)),
                       showarrow=False,row=1,col=(i+1))

fig.add_trace(go.Scatter(x=df2[df2['_source.data.row_index']==0]['_source.data.offset_pos_x_um'],y=df2[df2['_source.data.row_index']==0]['row_col'],
                         name="X-Offset                      ",mode="markers",marker=dict(color="MediumPurple")),row=1,col=1)
    
fig.update_layout(title="<b>X-Offset Plot for "+df2['_source.data.scanner_name'].unique()[0]+" "+str(df2['date'].unique()[-1]),height=800,width=1400)
fig.update_yaxes(autorange="reversed")
fig.update_yaxes(title="Slot Position",row=1,col=1)
fig.update_xaxes(title="X-Offset(um)",range=[-3550,3550])
fig.add_vline(x=-3500, line_width=3, line_dash="dash", line_color="red")
fig.add_vline(x=3500, line_width=3, line_dash="dash", line_color="red")
fig.add_annotation(x=1.158,y=0.9,xref="paper",yref="paper",
        text="<br><b>Basket Level Details <br>"+"<b>+ve Adjustments : "+str(len(df2[df2['_source.data.offset_pos_x_um']>0]))+\
                   "<br>-ve Adjustments : "+str(len(df2[df2['_source.data.offset_pos_x_um']<0]))+\
                       "<br>μ +ve Adjustments : "+str(round(np.mean(df2[df2['_source.data.offset_pos_x_um']>0]['_source.data.offset_pos_x_um']),2))+\
                       "<br>μ -ve Adjustments :"+str(round(np.mean(df2[df2['_source.data.offset_pos_x_um']<0]['_source.data.offset_pos_x_um']),2)),
        showarrow=False,font=dict(family="Courier New, monospace",size=10,color="black"),align="center",
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="white",
        opacity=0.8
        )
fig.show()


# Y OFFSET

fig = make_subplots(
            rows=4, cols=1,
            subplot_titles=("<b>Row : 1 ","<b>Row : 2 ","<b>Row : 3 ","<b>Row : 4 "))
for i in range(4):
    fig.add_trace(go.Scatter(y=df2[df2['_source.data.row_index']==i]['_source.data.offset_pos_y_um'],x=df2[df2['_source.data.row_index']==i]['row_col'],
                             name="X-Offset",showlegend=False,mode="markers",marker=dict(color="MediumPurple")),row=(i+1),col=1)
    
    fig.add_annotation(x=-4,xref="x",yref="y",text="<b>Postive Adjustments : "+str(len(df2[(df2['_source.data.row_index']==i)&(df2['_source.data.offset_pos_y_um']>0)]))+\
                       "<br>μ Postive Adjustments : "+str(round(np.mean(df2[(df2['_source.data.row_index']==i)&(df2['_source.data.offset_pos_y_um']>0)]['_source.data.offset_pos_y_um']),2)),
                       showarrow=False,row=(i+1),col=1)

fig.add_trace(go.Scatter(y=df2[df2['_source.data.row_index']==0]['_source.data.offset_pos_y_um'],x=df2[df2['_source.data.row_index']==0]['row_col'],
                         name="Y-Offset                ",mode="markers",marker=dict(color="MediumPurple")),row=1,col=1)
    
fig.update_layout(title="<b>Y-Offset Plot for "+df2['_source.data.scanner_name'].unique()[0]+" "+str(df2['date'].unique()[-1]),height=800,width=1400)
# fig.update_xaxes(autorange="reversed")
fig.update_xaxes(tickangle=55)
fig.update_xaxes(title="Slot Position",row=4,col=1)
fig.update_yaxes(title="Y-Offset(um)",range=[0,5050])
fig.add_hline(y=0, line_width=3, line_dash="dash", line_color="red")
fig.add_hline(y=5000, line_width=3, line_dash="dash", line_color="red")
fig.add_annotation(x=1.158,y=0.9,xref="paper",yref="paper",
        text="<br><b>Basket Level Details <br>"+"<b>+ve Adjustments : "+str(len(df2[df2['_source.data.offset_pos_y_um']>0]))+\
                       "<br>μ +ve Adjustments : "+str(round(np.mean(df2[df2['_source.data.offset_pos_y_um']>0]['_source.data.offset_pos_y_um']),2)),
        showarrow=False,font=dict(family="Courier New, monospace",size=10,color="black"),align="center",
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="white",
        opacity=0.8
        )
fig.show()

In [ ]:
#SLIDE LOCKING PLOT

slide_locking = slide_locking.dropna(subset=['_source.data.row_index','_source.data.col_index'])
slide_locking['_source.data.row_index'] = slide_locking['_source.data.row_index'].astype(int)
slide_locking['_source.data.col_index'] = slide_locking['_source.data.col_index'].astype(int)
slide_locking['date'] = pd.to_datetime(slide_locking['_source.data.time_stamp']).dt.date

slide_locking['row_col'] = (slide_locking['_source.data.row_index']+1).astype(str)+"_"+(slide_locking['_source.data.col_index']+1).astype(str)
slide_locking = slide_locking.sort_values(["_source.data.row_index","_source.data.col_index"], ascending = (True, True))

# lst = ["CS006-1-1647954319260","CS006-2-1647954319268","CS006-3-1647954319272","CS006-4-1647954319277"]

# for i in load:
df2 = slide_locking[slide_locking['_source.data.load_identifier']== load]
# df2 = slide_locking[slide_locking['_source.data.scanner_name'] == 'H01EBB38P']
print(len(df2))

fig = px.scatter(df2,x='row_col',y="_source.data.first_current_diff",marginal_y="violin").update_traces(marker=dict(color="green"))
fig.update_layout(title="Current Difference Plot for : "+df2['_source.data.scanner_name'].unique()[0],height=600,width=1200,yaxis1=dict(title="Current Difference(mA)"),xaxis1=dict(title="Slot Position"))
fig.update_xaxes(tickangle=55)
#     fig.update_yaxes(range=[200,450])
#     fig.add_hline(y=150, line_width=3, line_dash="dash", line_color="red")   
fig.show()

In [ ]:
#PICKING ANGLE PLOT

slide_picking_from_scanner = slide_picking_from_scanner.dropna(subset=['_source.data.row_index','_source.data.col_index'])
slide_picking_from_scanner['_source.data.row_index'] = slide_picking_from_scanner['_source.data.row_index'].astype(int)
slide_picking_from_scanner['_source.data.col_index'] = slide_picking_from_scanner['_source.data.col_index'].astype(int)

slide_picking_from_scanner['row_col'] = (slide_picking_from_scanner['_source.data.row_index']+1).astype(str)+"_"+(slide_picking_from_scanner['_source.data.col_index']+1).astype(str)
slide_picking_from_scanner = slide_picking_from_scanner.sort_values(["_source.data.row_index","_source.data.col_index"], ascending = (True, True))

# lst = ["CS006-1-1647954319260","CS006-2-1647954319268","CS006-3-1647954319272","CS006-4-1647954319277"]

# for i in load:
df2 = slide_picking_from_scanner[slide_picking_from_scanner['_source.data.load_identifier'] == load]
print(len(df2))
fig = px.scatter(df2,x='row_col',y="_source.data.actual_angle",marginal_y="violin").update_traces(marker=dict(color="green"))
fig.update_layout(showlegend=False,title="Pickup Angle Plot for : "+df2['_source.data.scanner_name'].unique()[0],height=600,width=1200,yaxis1=dict(title="Current Difference(mA)"),xaxis1=dict(title="Slot Position"))
fig.update_xaxes(tickangle=55)
fig.add_hline(y=-1.5, line_width=3, line_dash="dash", line_color="red")
fig.add_hline(y=1.5, line_width=3, line_dash="dash", line_color="red")
fig.add_scatter(x=df2[df2['_source.data.actual_angle']>=1.5]['row_col'],y=df2[df2['_source.data.actual_angle']>=1.5]['_source.data.actual_angle'],marker=dict(color="red",size=12),mode="markers")
fig.add_scatter(x=df2[df2['_source.data.actual_angle']<=-1.5]['row_col'],y=df2[df2['_source.data.actual_angle']<=-1.5]['_source.data.actual_angle'],marker=dict(color="red",size=12),mode="markers")
fig.update_yaxes(range=[-10,10])
fig.show()
# df2

In [ ]:
# THICKNESS PLOT

basket_data = basket_data.dropna(subset=['_source.data.row_index','_source.data.col_index'])
basket_data['_source.data.row_index'] = basket_data['_source.data.row_index'].astype(int)
basket_data['_source.data.col_index'] = basket_data['_source.data.col_index'].astype(int)
basket_data['row_col'] = (basket_data['_source.data.row_index']+1).astype(str)+"_"+(basket_data['_source.data.col_index']+1).astype(str)
basket_data = basket_data.sort_values(["_source.data.row_index","_source.data.col_index"], ascending = (True, True))

# lst = ["CS006-1-1647954319260","CS006-2-1647954319268","CS006-3-1647954319272","CS006-4-1647954319277"]

df3 = basket_data[basket_data['_source.data.load_identifier'] == load]
df3
print(len(df3))
fig = px.scatter(df3, y='row_col',x="_source.data.slide_thickness",color="_source.data.scanner_name",marginal_x="histogram")
fig.update_yaxes(title='')
fig.update_xaxes(title='')
fig.update_layout(legend_title="Scanner",title="Slide Thickness Plot",height=800,yaxis1=dict(title="Slot Position"),xaxis1=dict(title="Slide Thickness(mm)"))
# fig.write_image("image_"+df2['_source.data.scanner_name'].unique()[0]+".png")
fig.show()